In [ ]:
experiment_name = "2025.05.07-03_AnalisiSentimentConGPT4omini-2024-07-18_BATCH-API"

In [ ]:
import pickle
import pandas as pd
import glob
import os
import json
from IPython.display import clear_output

os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
clear_output()

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/input/sdg_full_dataset_no-sdg0_2024.07.11.xlsx"
output_data_path = base_path + "/../AnalisiTesti/data/output/"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
temp = 0.0
model_name="gpt-4o-mini-2024-07-18"

In [ ]:
output_dir = output_data_path + f"{experiment_name}/"

In [ ]:
!pip install -q anyio==3.7.1 openai --progress-bar off

In [ ]:
test_data = pd.read_excel(base_path + "/../AnalisiTesti/data/output/RisultatiSentiment_EtichetatturaManuale_DatasetDiTest_CON_LABEL_FINALE.xlsx").drop(columns=[
    'Unnamed: 0'
])
test_data

,text_id,text,sdg_true,label
0,210,The Brazilian Instituto de Pesquisa Economica ...,1,positive
1,251,While the growth effect predominated in practi...,1,neutral
2,441,"Nevertheless, the use of indices as a form of ...",1,positive
3,591,"If it is high, then public policies designed t...",1,positive
4,1250,Results for countries using national panels ad...,1,neutral
...,...,...,...,...
571,5801,It may be commonplace to state that the world ...,16,neutral
572,6119,Sexual citizenship and sexual rights scholarsh...,16,positive
573,6551,Legal pluralism is a phenomenon defined differ...,16,positive
574,6618,"This article discusses the extent, impacts, an...",16,negative


In [ ]:
data_sel = test_data#.iloc[:3]
data_sel

,text_id,text,sdg_true,label
0,210,The Brazilian Instituto de Pesquisa Economica ...,1,positive
1,251,While the growth effect predominated in practi...,1,neutral
2,441,"Nevertheless, the use of indices as a form of ...",1,positive
3,591,"If it is high, then public policies designed t...",1,positive
4,1250,Results for countries using national panels ad...,1,neutral
...,...,...,...,...
571,5801,It may be commonplace to state that the world ...,16,neutral
572,6119,Sexual citizenship and sexual rights scholarsh...,16,positive
573,6551,Legal pluralism is a phenomenon defined differ...,16,positive
574,6618,"This article discusses the extent, impacts, an...",16,negative


In [ ]:
# Esempio formato previsto per utilizzo Batch API:

{
    "custom_id": "request-1",
    "method": "POST",
    "url": "/v1/chat/completions",
    "body": {
        "model": "gpt-3.5-turbo-0125",
        "messages": [
            {
              "role": "system",
              "content": "You are a helpful assistant."
            },
            {
              "role": "user",
              "content": "Hello world!"
            }
        ],
        "max_tokens": 1000 # adesso e' max_completion_tokens
    }
}

{'custom_id': 'request-1',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-3.5-turbo-0125',
  'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
   {'role': 'user', 'content': 'Hello world!'}],
  'max_tokens': 1000}}

In [ ]:
list_of_jsons_4_batch_api = []
for index, row in data_sel.iterrows():

    cnt = index
    text_id = row['text_id']
    sdg_true = row['sdg_true']
    text = row['text']

    template_sentiment = f"""Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

  - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

Return the result in JSON format with 1 key:

  - "label": The assigned label for the input text [this key/value pair is mandatory].


INPUT TEXT: \"{text}\"
SDG_CLASSIFICATION: \"SDG-{sdg_true}\"
OUTPUT: """

    template = template_sentiment

    json_4_batch_api = {
        "custom_id": f"request-sentiment-o4mini-{text_id}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": model_name,
            "messages": [
                {
                  "role": "system",
                  "content": "You are a helpful assistant."
                },
                {
                  "role": "user",
                  "content": template_sentiment
                }
            ],
            "max_completion_tokens": 1000
        }
    }

    list_of_jsons_4_batch_api.append(json_4_batch_api)

In [ ]:
len(list_of_jsons_4_batch_api)

576

In [ ]:
list_of_jsons_4_batch_api[0]

{'custom_id': 'request-sentiment-o4mini-210',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini-2024-07-18',
  'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
   {'role': 'user',
    'content': 'Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:\n\n  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.\n\n  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.\n\n  - "neutral": the text does not imply or explicitly state that what is affirmed or desc

In [ ]:
# Apertura del file in modalità scrittura
with open(output_dir+'list_of_jsons_4_batch_api_SENTIMENT_TASK.jsonl', 'w') as file:
    for dictionary in list_of_jsons_4_batch_api:
        # Conversione del dizionario in una stringa JSON
        json_string = json.dumps(dictionary)
        # Scrittura della stringa JSON nel file seguita da una nuova linea
        file.write(json_string + '\n')

In [ ]:
STOP!

SyntaxError: invalid syntax (<ipython-input-14-3bfd59f24194>, line 1)

Upload Batch File


In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
batch_input_file = client.files.create(
  file=open(output_dir+'list_of_jsons_4_batch_api_SENTIMENT_TASK.jsonl', "rb"),
  purpose="batch"
)

In [ ]:
client.files.list()

SyncCursorPage[FileObject](data=[FileObject(id='file-6rhj4U14PJAZucmiFEGz9N', bytes=1146844, created_at=1746632417, filename='list_of_jsons_4_batch_api_SENTIMENT_TASK.jsonl', object='file', purpose='batch', status='processed', status_details=None, expires_at=1749224417), FileObject(id='file-61dcxwvcmokK19y11kvYxd', bytes=512279, created_at=1746630341, filename='batch_681b6aded5308190b0a4141b285f3fd8_output.jsonl', object='file', purpose='batch_output', status='processed', status_details=None, expires_at=1749222341), FileObject(id='file-VSvZ9VV6GVGNHjGYBMWa3w', bytes=1159516, created_at=1746627255, filename='list_of_jsons_4_batch_api_SENTIMENT_TASK.jsonl', object='file', purpose='batch', status='processed', status_details=None, expires_at=1749219255), FileObject(id='file-XHkJ2p2LoU95PZDVwWB51s', bytes=47812, created_at=1746621679, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None, expires_at=None), FileObject(id='file-CLwuXJ

In [ ]:
batch_input_file_id = 'file-6rhj4U14PJAZucmiFEGz9N' # file_id for list_of_jsons_4_batch_api_SENTIMENT_TASK.jsonl

In [ ]:
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Batch run for polarity detection task on SDG test dataset (576 texts) with GPT4omini-2024-07-18"
    }
)

Batch(id='batch_681b7f0daf788190887dbff06101ef7b', completion_window='24h', created_at=1746632461, endpoint='/v1/chat/completions', input_file_id='file-6rhj4U14PJAZucmiFEGz9N', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1746718861, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch run for polarity detection task on SDG test dataset (576 texts) with GPT4omini-2024-07-18'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
client.batches.retrieve("batch_681b7f0daf788190887dbff06101ef7b")

Batch(id='batch_681b7f0daf788190887dbff06101ef7b', completion_window='24h', created_at=1746632461, endpoint='/v1/chat/completions', input_file_id='file-6rhj4U14PJAZucmiFEGz9N', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1746718861, failed_at=None, finalizing_at=None, in_progress_at=1746632463, metadata={'description': 'Batch run for polarity detection task on SDG test dataset (576 texts) with GPT4omini-2024-07-18'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=576))

In [ ]:
client.batches.retrieve("batch_681b7f0daf788190887dbff06101ef7b")

Batch(id='batch_681b7f0daf788190887dbff06101ef7b', completion_window='24h', created_at=1746632461, endpoint='/v1/chat/completions', input_file_id='file-6rhj4U14PJAZucmiFEGz9N', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1746718861, failed_at=None, finalizing_at=None, in_progress_at=1746632463, metadata={'description': 'Batch run for polarity detection task on SDG test dataset (576 texts) with GPT4omini-2024-07-18'}, output_file_id=None, request_counts=BatchRequestCounts(completed=541, failed=0, total=576))

In [ ]:
client.batches.retrieve("batch_681b7f0daf788190887dbff06101ef7b")

Batch(id='batch_681b7f0daf788190887dbff06101ef7b', completion_window='24h', created_at=1746632461, endpoint='/v1/chat/completions', input_file_id='file-6rhj4U14PJAZucmiFEGz9N', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1746633580, error_file_id=None, errors=None, expired_at=None, expires_at=1746718861, failed_at=None, finalizing_at=1746633521, in_progress_at=1746632463, metadata={'description': 'Batch run for polarity detection task on SDG test dataset (576 texts) with GPT4omini-2024-07-18'}, output_file_id='file-Vkw7wfNdpTZQDcQz5FQKo3', request_counts=BatchRequestCounts(completed=576, failed=0, total=576))

In [ ]:
file_response = client.files.content("file-Vkw7wfNdpTZQDcQz5FQKo3")
print(file_response.text)

{"id": "batch_req_681b833e8e1c81908557a4606064a3d4", "custom_id": "request-sentiment-o4mini-210", "response": {"status_code": 200, "request_id": "9f91b90c6821564116f864b28225e4e2", "body": {"id": "chatcmpl-BUbDjWvPoKjfopUAsgi7kgrMelFmj", "object": "chat.completion", "created": 1746632615, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "```json\n{\n  \"label\": \"positive\"\n}\n```", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 338, "completion_tokens": 14, "total_tokens": 352, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_dbaca60df0"}}, "error": null}
{"id": "batch_req_681b833ea0e88190a6297b5b964e6d03", "custom_id": "request-sentiment-o4mini-251", "response"

In [ ]:
# dir(file_response)

In [ ]:
file_response.write_to_file(output_dir+"POLARITYDETECTION_576_TESTI_GPTo4mini-2024-07-18_BATCH_API.jsonl")